# In this notebook, we will apply data generator to monitor test outcome on Grafana dashboard.

In [1]:
# TensorFlow and tf.keras to load the data
import tensorflow as tf
from tensorflow import keras

# Commonly used modules
import numpy as np
import os
import sys

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd


import IPython
from six.moves import urllib

print(tf.__version__)

/home/roshan/anaconda3/envs/monitoring/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/roshan/anaconda3/envs/monitoring/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/roshan/anaconda3/envs/monitoring/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/roshan/anaconda3/envs/monito

1.14.0


# The data set is Boston Housing Prices classification data set https://www.kaggle.com/vikrishnan/boston-house-prices
## Step 1: Pre-process the data (normalization)

In [2]:
(train_features, train_labels), (test_features, test_labels) = keras.datasets.boston_housing.load_data()

# get per-feature statistics (mean, standard deviation) from the training set to normalize by
train_mean = np.mean(train_features, axis=0)
train_std = np.std(train_features, axis=0)
train_features = (train_features - train_mean) / train_std
print("Number of features per sample=", np.shape(train_features)[1])

Number of features per sample= 13


In [3]:
# Do the same for test features
test_mean = np.mean(test_features, axis=0)
test_std = np.std(test_features, axis=0)
test_features = (test_features - test_mean) / test_std

## Step 2: Define a simple Neural network model

In [4]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

In [5]:
def nn_model():
    model = keras.Sequential([
        Dense(100, activation=tf.nn.relu, input_shape=[len(train_features[0])]),
        Dense(1)
    ])

    model.compile(optimizer='adam', 
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model

## Next, lets train the model

In [6]:
#Now, lets train the model
model = nn_model()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(train_features, train_labels, epochs=1000, verbose=1, validation_split = 0.1,
                    callbacks=[early_stop])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 363 samples, validate on 41 samples


2021-12-08 03:52:51.180973: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-12-08 03:52:51.461823: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394530000 Hz
2021-12-08 03:52:51.462460: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5631e9c32130 executing computations on platform Host. Devices:
2021-12-08 03:52:51.462489: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2021-12-08 03:52:51.882567: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not 

Epoch 1/1000
363/363 [==============================] - 1s 3ms/sample - loss: 593.0693 - mean_absolute_error: 22.4235 - mean_squared_error: 593.0692 - val_loss: 487.8278 - val_mean_absolute_error: 21.0906 - val_mean_squared_error: 487.8278
Epoch 2/1000
363/363 [==============================] - 0s 40us/sample - loss: 567.9838 - mean_absolute_error: 21.8941 - mean_squared_error: 567.9839 - val_loss: 465.5730 - val_mean_absolute_error: 20.5722 - val_mean_squared_error: 465.5730
Epoch 3/1000
363/363 [==============================] - 0s 56us/sample - loss: 543.7433 - mean_absolute_error: 21.3702 - mean_squared_error: 543.7433 - val_loss: 443.0162 - val_mean_absolute_error: 20.0414 - val_mean_squared_error: 443.0161
Epoch 4/1000
363/363 [==============================] - 0s 43us/sample - loss: 517.7377 - mean_absolute_error: 20.8124 - mean_squared_error: 517.7378 - val_loss: 419.4012 - val_mean_absolute_error: 19.4655 - val_mean_squared_error: 419.4012
Epoch 5/1000
363/363 [===============

363/363 [==============================] - 0s 47us/sample - loss: 29.7144 - mean_absolute_error: 3.8312 - mean_squared_error: 29.7144 - val_loss: 25.1076 - val_mean_absolute_error: 4.0225 - val_mean_squared_error: 25.1076
Epoch 36/1000
363/363 [==============================] - 0s 46us/sample - loss: 28.7990 - mean_absolute_error: 3.7707 - mean_squared_error: 28.7990 - val_loss: 24.9717 - val_mean_absolute_error: 3.9998 - val_mean_squared_error: 24.9717
Epoch 37/1000
363/363 [==============================] - 0s 50us/sample - loss: 27.9289 - mean_absolute_error: 3.7227 - mean_squared_error: 27.9289 - val_loss: 24.8290 - val_mean_absolute_error: 3.9769 - val_mean_squared_error: 24.8290
Epoch 38/1000
363/363 [==============================] - 0s 43us/sample - loss: 27.1605 - mean_absolute_error: 3.6761 - mean_squared_error: 27.1605 - val_loss: 24.4990 - val_mean_absolute_error: 3.9299 - val_mean_squared_error: 24.4990
Epoch 39/1000
363/363 [==============================] - 0s 51us/sampl

Epoch 70/1000
363/363 [==============================] - 0s 45us/sample - loss: 16.7335 - mean_absolute_error: 2.9107 - mean_squared_error: 16.7335 - val_loss: 18.4842 - val_mean_absolute_error: 3.2132 - val_mean_squared_error: 18.4842
Epoch 71/1000
363/363 [==============================] - 0s 51us/sample - loss: 16.5815 - mean_absolute_error: 2.8991 - mean_squared_error: 16.5815 - val_loss: 18.3292 - val_mean_absolute_error: 3.1935 - val_mean_squared_error: 18.3292
Epoch 72/1000
363/363 [==============================] - 0s 39us/sample - loss: 16.4068 - mean_absolute_error: 2.8769 - mean_squared_error: 16.4068 - val_loss: 18.3864 - val_mean_absolute_error: 3.2160 - val_mean_squared_error: 18.3864
Epoch 73/1000
363/363 [==============================] - 0s 51us/sample - loss: 16.1369 - mean_absolute_error: 2.8496 - mean_squared_error: 16.1369 - val_loss: 18.1914 - val_mean_absolute_error: 3.1944 - val_mean_squared_error: 18.1914
Epoch 74/1000
363/363 [==============================] -

...

363/363 [==============================] - 0s 45us/sample - loss: 6.7056 - mean_absolute_error: 1.7938 - mean_squared_error: 6.7056 - val_loss: 5.7967 - val_mean_absolute_error: 2.0579 - val_mean_squared_error: 5.7967
Epoch 281/1000
363/363 [==============================] - 0s 52us/sample - loss: 6.6482 - mean_absolute_error: 1.7883 - mean_squared_error: 6.6482 - val_loss: 5.4213 - val_mean_absolute_error: 1.9717 - val_mean_squared_error: 5.4213
Epoch 282/1000
363/363 [==============================] - 0s 44us/sample - loss: 6.6609 - mean_absolute_error: 1.8038 - mean_squared_error: 6.6609 - val_loss: 5.4055 - val_mean_absolute_error: 1.9638 - val_mean_squared_error: 5.4055
Epoch 283/1000
363/363 [==============================] - 0s 47us/sample - loss: 6.6738 - mean_absolute_error: 1.8070 - mean_squared_error: 6.6738 - val_loss: 5.6513 - val_mean_absolute_error: 2.0244 - val_mean_squared_error: 5.6513
Epoch 284/1000
363/363 [==============================] - 0s 48us/sample - loss: 6.

363/363 [==============================] - 0s 51us/sample - loss: 6.2562 - mean_absolute_error: 1.7519 - mean_squared_error: 6.2562 - val_loss: 5.3160 - val_mean_absolute_error: 1.9448 - val_mean_squared_error: 5.3160
Epoch 316/1000
363/363 [==============================] - 0s 48us/sample - loss: 6.2010 - mean_absolute_error: 1.7413 - mean_squared_error: 6.2010 - val_loss: 5.5194 - val_mean_absolute_error: 2.0003 - val_mean_squared_error: 5.5194
Epoch 317/1000
363/363 [==============================] - 0s 46us/sample - loss: 6.2756 - mean_absolute_error: 1.7369 - mean_squared_error: 6.2756 - val_loss: 5.6785 - val_mean_absolute_error: 2.0340 - val_mean_squared_error: 5.6785
Epoch 318/1000
363/363 [==============================] - 0s 45us/sample - loss: 6.2164 - mean_absolute_error: 1.7352 - mean_squared_error: 6.2164 - val_loss: 5.1131 - val_mean_absolute_error: 1.9073 - val_mean_squared_error: 5.1131
Epoch 319/1000
363/363 [==============================] - 0s 49us/sample - loss: 6.

363/363 [==============================] - 0s 43us/sample - loss: 5.8584 - mean_absolute_error: 1.6717 - mean_squared_error: 5.8584 - val_loss: 5.3222 - val_mean_absolute_error: 1.9327 - val_mean_squared_error: 5.3222
Epoch 351/1000
363/363 [==============================] - 0s 47us/sample - loss: 5.8567 - mean_absolute_error: 1.6771 - mean_squared_error: 5.8567 - val_loss: 5.0913 - val_mean_absolute_error: 1.8690 - val_mean_squared_error: 5.0913
Epoch 352/1000
363/363 [==============================] - 0s 47us/sample - loss: 5.8479 - mean_absolute_error: 1.6763 - mean_squared_error: 5.8479 - val_loss: 5.2935 - val_mean_absolute_error: 1.9168 - val_mean_squared_error: 5.2935
Epoch 353/1000
363/363 [==============================] - 0s 49us/sample - loss: 5.8923 - mean_absolute_error: 1.6805 - mean_squared_error: 5.8923 - val_loss: 5.1785 - val_mean_absolute_error: 1.9149 - val_mean_squared_error: 5.1785
Epoch 354/1000
363/363 [==============================] - 0s 54us/sample - loss: 5.

363/363 [==============================] - 0s 46us/sample - loss: 5.5153 - mean_absolute_error: 1.6257 - mean_squared_error: 5.5153 - val_loss: 5.1289 - val_mean_absolute_error: 1.8676 - val_mean_squared_error: 5.1289
Epoch 386/1000
363/363 [==============================] - 0s 42us/sample - loss: 5.4804 - mean_absolute_error: 1.6211 - mean_squared_error: 5.4804 - val_loss: 5.1862 - val_mean_absolute_error: 1.8460 - val_mean_squared_error: 5.1862
Epoch 387/1000
363/363 [==============================] - 0s 55us/sample - loss: 5.5041 - mean_absolute_error: 1.6196 - mean_squared_error: 5.5041 - val_loss: 5.2013 - val_mean_absolute_error: 1.8698 - val_mean_squared_error: 5.2013
Epoch 388/1000
363/363 [==============================] - 0s 54us/sample - loss: 5.4990 - mean_absolute_error: 1.6207 - mean_squared_error: 5.4990 - val_loss: 5.1002 - val_mean_absolute_error: 1.8331 - val_mean_squared_error: 5.1002
Epoch 389/1000
363/363 [==============================] - 0s 43us/sample - loss: 5.

363/363 [==============================] - 0s 47us/sample - loss: 5.1827 - mean_absolute_error: 1.5719 - mean_squared_error: 5.1827 - val_loss: 5.1152 - val_mean_absolute_error: 1.7981 - val_mean_squared_error: 5.1152
Epoch 421/1000
363/363 [==============================] - 0s 43us/sample - loss: 5.1878 - mean_absolute_error: 1.5775 - mean_squared_error: 5.1878 - val_loss: 5.3057 - val_mean_absolute_error: 1.8294 - val_mean_squared_error: 5.3057
Epoch 422/1000
363/363 [==============================] - 0s 41us/sample - loss: 5.1700 - mean_absolute_error: 1.5683 - mean_squared_error: 5.1700 - val_loss: 5.1632 - val_mean_absolute_error: 1.7891 - val_mean_squared_error: 5.1632
Epoch 423/1000
363/363 [==============================] - 0s 49us/sample - loss: 5.2351 - mean_absolute_error: 1.5917 - mean_squared_error: 5.2351 - val_loss: 5.0335 - val_mean_absolute_error: 1.7579 - val_mean_squared_error: 5.0335
Epoch 424/1000
363/363 [==============================] - 0s 46us/sample - loss: 5.

In [7]:
# show RMSE measure to compare to Kaggle leaderboard on https://www.kaggle.com/c/boston-housing/leaderboard
rmse_final = np.sqrt(float(hist['val_loss'].tail(1)))
print()
print('Final Root Mean Square Error on validation set: {}'.format(round(rmse_final, 3)))


Final Root Mean Square Error on validation set: 2.283


In [8]:
print(hist)

           loss  mean_absolute_error  mean_squared_error    val_loss  \
0    593.069256            22.423489          593.069214  487.827791   
1    567.983838            21.894096          567.983887  465.572952   
2    543.743288            21.370195          543.743347  443.016162   
3    517.737744            20.812422          517.737793  419.401202   
4    491.449619            20.223795          491.449585  393.625732   
..          ...                  ...                 ...         ...   
446    4.980011             1.536222            4.980011    5.317154   
447    5.037918             1.541982            5.037918    5.281909   
448    5.004770             1.551956            5.004770    5.285040   
449    4.968571             1.529417            4.968572    5.317424   
450    4.968346             1.526944            4.968346    5.214257   

     val_mean_absolute_error  val_mean_squared_error  epoch  
0                  21.090593              487.827789      0  
1          

In [9]:
mse, _, _ = model.evaluate(test_features, test_labels)
rmse = np.sqrt(mse)
print('Root Mean Square Error on test set: {}'.format(round(rmse, 3)))

102/102 [==============================] - 0s 44us/sample - loss: 14.8341 - mean_absolute_error: 2.6423 - mean_squared_error: 14.8341
Root Mean Square Error on test set: 3.852


## Now, lets generate more test samples from test data and monitor them, as priduction data sets

In [10]:
print(np.shape(train_features), np.shape(train_labels))
print(np.shape(test_features), np.shape(test_labels))

(404, 13) (404,)
(102, 13) (102,)


In [11]:
num=np.shape(test_labels)[0]
import random

In [12]:
#Lets import the ml_monitor library
import ml_monitor
import time
import numpy as np

In [13]:
my_monitor = ml_monitor.Monitor()
my_monitor.start()

2021-12-08 03:54:20,006 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 03:54:25,007 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 03:54:30,011 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...


In [15]:
epochs=10000000
for i in range(epochs):
    indx=random.randint(0,num-1)
    x_batch=np.expand_dims(test_features[indx,:],axis=0)
    y_batch=np.expand_dims(test_labels[indx],axis=0)
    mse, _, _ = model.evaluate(x_batch, y_batch, verbose=0)
    my_monitor.monitor("n_rmse", np.sqrt(mse))

2021-12-08 04:03:15,317 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:03:20,322 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:03:25,327 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:03:30,330 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:03:35,333 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:03:40,336 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:03:45,347 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:03:50,350 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:03:55,354 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:04:00,356 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:04:05,358 [10802] INFO    

2021-12-08 04:10:25,662 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:10:30,665 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:10:35,668 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:10:40,674 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:10:45,675 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:10:50,679 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:10:55,685 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:11:00,688 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:11:05,692 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:11:10,695 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:11:15,699 [10802] INFO    

2021-12-08 04:17:35,965 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:17:40,968 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:17:45,972 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:17:50,974 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:17:55,978 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:18:00,982 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:18:05,985 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:18:10,988 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:18:15,990 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:18:20,996 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:18:26,000 [10802] INFO    

2021-12-08 04:24:46,276 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:24:51,280 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:24:56,284 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:25:01,289 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:25:06,293 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:25:11,296 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:25:16,300 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:25:21,303 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:25:26,308 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:25:31,311 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:25:36,316 [10802] INFO    

2021-12-08 04:31:56,599 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:32:01,604 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:32:06,605 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:32:11,610 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:32:16,613 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:32:21,617 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:32:26,621 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:32:31,624 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:32:36,627 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:32:41,631 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 04:32:46,632 [10802] INFO    

...

2021-12-08 10:59:13,428 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 10:59:18,432 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 10:59:23,435 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 10:59:28,438 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 10:59:33,441 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 10:59:38,444 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 10:59:43,447 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 10:59:48,451 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 10:59:53,454 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 10:59:58,457 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:00:03,460 [10802] INFO    

2021-12-08 11:06:23,689 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:06:28,692 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:06:33,695 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:06:38,698 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:06:43,701 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:06:48,704 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:06:53,707 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:06:58,710 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:07:03,714 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:07:08,717 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:07:13,720 [10802] INFO    

2021-12-08 11:13:33,928 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:13:38,930 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:13:43,933 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:13:48,934 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:13:53,938 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:13:58,941 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:14:03,944 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:14:08,948 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:14:13,951 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:14:18,952 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:14:23,956 [10802] INFO    

2021-12-08 11:20:44,163 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:20:49,166 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:20:54,169 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:20:59,171 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:21:04,173 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:21:09,175 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:21:14,176 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:21:19,179 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:21:24,182 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:21:29,184 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:21:34,187 [10802] INFO    

2021-12-08 11:27:54,393 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:27:59,397 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:28:04,399 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:28:09,402 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:28:14,405 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:28:19,407 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:28:24,410 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:28:29,413 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:28:34,416 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:28:39,418 [10802] INFO     ml_monitor.logging: Starting metrics logging thread...
2021-12-08 11:28:44,421 [10802] INFO    